In [ ]:
import numpy as np
import cv2
import shutil
import os
from utils import make_dirs, BoxesGen, Encoder, SSDAugmentation
from model import SSD
from dataloader import collate_fn
from  PIL import Image
import torch
import pandas as pd


In [ ]:
classes = ["background", "bicycle", "car", "motorcycle", "bus"] 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = SSD()
classes = ["background", "bicycle", "car", "motorcycle", "bus"] 
colors = [None, (164, 80, 133), (39, 129, 113), (83, 122, 114), (99, 81, 172)]
batch_size = 1
num_workers = 4
nms_threshold = 0.5
cls_threshold = 0.5
pretrained_model = ""
output_path = "output"
input_path = "/mnt/BA5672185671D59B/RCNN/object_detection_old/data/objects/test"
result_df = pd.DataFrame({}, columns=['filename', 'class', 'id', 'coordinates'])

checkpoint = torch.load(pretrained_model)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)
model.eval()
dboxes = BoxesGen()
encoder = Encoder(dboxes)
transformer = SSDAugmentation(dboxes, (300, 300), val=True)
data_path = ""

test_params = {"batch_size": batch_size, "shuffle": False,
                "drop_last": False, "num_workers": num_workers,
                "collate_fn": collate_fn}
#     test_set = CocoDataset(opt.data_path, 2017, "val", SSDTransformer(dboxes, (300, 300), val=True))
import shutil
if os.path.isdir(output_path):
    shutil.rmtree(output_path)
make_dirs(output_path)


In [ ]:
input_path = "/mnt/BA5672185671D59B/RCNN/object_detection_old/data/objects/test"

In [ ]:
def annotate_imgs(loc, label, pred_score, output_path, output_img):
    if len(loc) > 0:
        loc[:, 0::2] *= width
        loc[:, 1::2] *= height
        loc = loc.astype(np.int32)
        for box, lb, pr in zip(loc, label, pred_score):
            category = classes[lb]
            color = colors[lb]
            xmin, ymin, xmax, ymax = box
            filename_l.append(image)
            class_l.append(category)
            id_l.append(lb)
            coord_l.append([xmin, ymin, xmax, ymax])
            cv2.rectangle(output_img, (xmin, ymin), (xmax, ymax), color, 2)
            text_size = cv2.getTextSize(category + " : %.2f" % pr, cv2.FONT_HERSHEY_PLAIN, 1, 1)[0]
            cv2.rectangle(output_img, (xmin, ymin), (xmin + text_size[0] + 3, ymin + text_size[1] + 4), color, -1)
            cv2.putText(output_img, category + " : %.2f" % pr,
                        (xmin, ymin + text_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1,
                        (255, 255, 255), 1)
        if output is None:
            output = "{}_res.jpg".format(output_path[:-4])
        else:
            output = output
        cv2.imwrite(output, output_img)

In [ ]:
filename_l = []
class_l = []
id_l = []
coord_l = []
list_images = os.listdir(input_path)
for image in list_images:
    with torch.no_grad():
        output_img = Image.open(os.path.join(input_path, image)).convert("RGB")
        # print(os.path.join(input_path, image))
        img = output_img.copy()
        height, width = img.size[0], img.size[1]
        img, _, _, _ = transformer(img, None, torch.zeros(1,4), torch.zeros(1))
        img = img.unsqueeze(dim=0)
        img = img.to(device)
        pred_box, pred_lbl = model(img)
        result = encoder.decode_batch(pred_box, pred_lbl, nms_threshold, 20)[0]
        loc, label, pred_score = [r.cpu().numpy() for r in result]
        best = np.argwhere(pred_score > cls_threshold).squeeze(axis=1)
        loc, label, pred_score = loc[best], label[best], pred_score[best]
        annotate_imgs(loc, label, pred_score, output_img, output_path)
        
result_df['filename'] = filename_l
result_df['class'] = class_l
result_df['id'] = id_l
result_df['coordinates'] = coord_l
result_df.to_csv('result_submission.csv', index = False)